# A Simple Contract

In this notebook we will learn to create the simplest possible financial contract as a qablet timetable, and price it with a deterministic model. We will use the following packages.

- `qablet_contracts` contains utilities to create qablet timetables for financial contracts.
- `qablet` contains a suite of models to evaluate qablet timetables. In this notebook we will use `FixedModel`.

To use these packages in your local environment, you will need `pip install qablet-contracts` and `pip install qablet-basic`. For complete API documentation see [qablet.github.io/qablet-contracts](https://qablet.github.io/qablet-contracts/) and [qablet-academy.github.io/intro/](https://qablet-academy.github.io/intro/).

Let us start with the imports.

In [ ]:
import numpy as np
from qablet_contracts.timetable import timetable_from_dicts
from qablet.base.fixed import FixedModel

## Create the contract

A contract is described by a list of events. We will create our first contract using a single event. An event has five properties:
- track
- time
- op
- quantity
- and unit

The event below is an example of a `+` **op**, which pays **quantity** 100, at **time** 1.0. In later notebooks we will learn about tracks, units, and other ops.


In [ ]:

events = [
    {
        "track": "",
        "time": 1.0,
        "op": "+",
        "quantity": 100.0,
        "unit": "USD"
    },
]
timetable = timetable_from_dicts(events)
print(timetable["events"].to_pandas())

  track  time op  quantity unit
0         1.0  +     100.0  USD


The utility `timetable_from_dicts` creates the timetable in the format expected by qablet pricing models. The only component relevant for our simple example is *events* , which is a [pyarrow table](https://arrow.apache.org/docs/python/generated/pyarrow.Table.html).

## Create a Dataset to price the contract
For our simple contract, the following data is needed
- BASE. The name of the currency in which the contract will be valued.
- ASSETS. The forward or discount curve for all assets required for valuation. We will define the asset `USD` using an array of times (years) and zero rates (fraction).

`FixedModel` needs nothing else. Other models will reuire model specific parameters in the dataset, as we will see in other notebooks. For more details see [Dataset API](https://qablet-academy.github.io/intro/dataset/)

In [ ]:

times = np.array([0.0, 1.0, 2.0, 5.0])
rates = np.array([0.04, 0.04, 0.045, 0.05])  # i.e. 4%, 4%, 4.5%, 5%
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

dataset = {
    "BASE": "USD",
    "ASSETS": {"USD": discount_data},
}

## Calculate Price

In [ ]:

model = FixedModel()
price, _ = model.price(timetable, dataset)
print(f"price: {price:11.6f}")


price:   96.078944


## Contract with multiple events

A sequence of payments is represented by a sequence of `+` events on the same track. In this example we construct a three year bond, paying a 5% coupon semi-annually. 

In [ ]:
maturity = 3
freq = 2  # semi-annual
coupon = 5 / freq

# Interest Payments
events = [
    {"track": "",
     "time": i / freq,
     "op": "+",
     "quantity": coupon,
     "unit": "USD"
    } for i in range(1, freq * maturity + 1)]

# Principal
events.append({"track": "", "time": maturity, "op": "+", "quantity": 100.0, "unit": "USD"})

timetable = timetable_from_dicts(events)
print(timetable["events"].to_pandas())

  track  time op  quantity unit
0         0.5  +       2.5  USD
1         1.0  +       2.5  USD
2         1.5  +       2.5  USD
3         2.0  +       2.5  USD
4         2.5  +       2.5  USD
5         3.0  +       2.5  USD
6         3.0  +     100.0  USD


We calculate the price as before.

In [ ]:
price, _ = model.price(timetable, dataset)
print(f"price: {price:11.6f}")

price:  100.517354


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bc6357ff-4a45-421b-9726-87fe96e1ac03' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>